## Multistorey House

Come per ogni homework ho scelto un immagine di riferimento:
![riferimento](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/lines_3b/base.png?raw=true)

Per quanto riguarda la progettazione della mia soluzione, ho iniziato definendomi a grosse linee come dovesse essere fatta la mia funzione multisorey_house(): essa al suo inerno richiamava altre sottofunzioni:

1. generate_intern_structure: che si occupa di creare gli interni prendendo come input le informazioni delle porte e delle finestre (da personalizzare a seconda delle esigenze) e la directory dove trovare i file tipici per la creazione di una struttura (vedere hw 8)
1. genrate_floors_stairs che di default è 1, si occupa di creare le scale qualora il numero di piani sia maggiore di uno, e di replicare la struttura passata come riferimento per il numero di piani richiesti
1. add_roof serve, per l'appunto, per generare il tetto. Riceve due informazioni: la prima è un flag che ci dice se la struttura ha un tetto, in caso contrario si accede ai dati realtivi a come costrutire un tetto (la seconda informazione passata come parametro) che devono essere passati con molta attenzione, per non creare tetti impossibili (vedere hw 09)

I problemi più grandi sono stati riscontrati nell'inserimento dei beam/pillar e della scala, che, al momento, può essere inserita di una sola tipologia però ouò essere spostata a piacimento all'interno della struttura. La scala viene creata a singola rampa, ed il soffitto viene bucato da una parte o dall'altra a seconda se ci si trova in piani pari o in piani dispari.



In [95]:
%run "my_function.ipynb"

In [96]:
def multisorey_house():
    def generate_intern_structure(doors_data,windows_data,dir_name="lines"):
        
        def genrate_floors_stairs(nFloor=1,external=1):
            def add_roof(hasRoof=0,roofData=[]):
            
                result=generate_structure(doors_data,windows_data,dir_name)
                first_floor=result[1]
                VIEW(first_floor)
                floor=result[0]
            
                beamStructureEven=generate_beam_structure(dir_name,floor,1)
                beamStructureOdd=generate_beam_structure(dir_name,floor,2)
            
                first_floorEven=(STRUCT([beamStructureEven,first_floor]))
                first_floorOdd=(STRUCT([beamStructureOdd,first_floor]))
              

                myFloors=[]
                myFloors.append(floor)
                myFloors.append(first_floorOdd)
                if(nFloor>1):
                    for i in range(1,nFloor):
                        stairsPos= findStair(dir_name)
                        stairsPos=SOLIDIFY(stairsPos)
                        stairsPos=OFFSET([2,2,2])(stairsPos)
                    
                        onZ=SIZE(3)(STRUCT([S([1,2,3])([0.1,0.1,0.1]),(first_floorEven)]))
                        onX=SIZE(1)(stairsPos)
                        onY=SIZE(2)(stairsPos)
                        mom= CUBOID([onX,onY,1])
                        mom=(STRUCT([S([1,2])([0.1,0.1]),(mom)]))
                        onX=SIZE(1)(mom)
                        onY=SIZE(2)(mom)
                        
                                        
                        stairs=ggpl_scale_a_rampa(onX-0.2,onY-0.2,onZ)
                        stairsR=STRUCT([T([1,2])([onX,onY]),R([1,2])(math.pi),stairs[0]])
                    
                        stairsR=STRUCT([S([1,2,3])([10,10,10]),stairsR])
                        stairs=STRUCT([S([1,2,3])([10,10,10]),stairs[0]])
                    
                        stairsPos=find_verts_and_cells("./"+dir_name+"/stairs.lines")
                        stairsPos=stairsPos[0]
                        if(i%2==1):
                            myFloors.append(T([1,2])([stairsPos[0][0],stairsPos[0][1]-1]))
                            myFloors.append(stairsR)
                            myFloors.append(T([2])([+1]))
                        else:
                            myFloors.append(T([1,2])([stairsPos[0][0]+1,stairsPos[0][1]+1]))
                            myFloors.append(stairs)
                            myFloors.append(T([1,2])([-1,-1]))
                    
                        myFloors.append(T([1,2])([-1*stairsPos[0][0],-1*stairsPos[0][1]]))
                        myFloors.append(T(3)(SIZE(3)(first_floorEven)))
                        if(i%2==1):
                            myFloors.append(first_floorEven)
                        else:
                            myFloors.append(first_floorOdd)
                             
                    
                result=STRUCT(myFloors)
                
                if(hasRoof==1):
                    roof_to_build=find_verts_and_cells("./"+dir_name+"/external_wall.lines")
                    roof_to_build=roof_to_build[0]
                    
                    
                    radiant=[PI/4]*roofData[0]
                    roof_cells=roofData[1]
                    rotation=roofData[2]
                    roof_height=roofData[3]
                    
                    map_point={}
                    roof_P=[]
                    
                    for i in range(0,len(roof_to_build)):
                        if(map_point.get((roof_to_build[i][0],roof_to_build[i][1]))== None):
                            roof_P.append(roof_to_build[i])
                            roof_to_build[i].append(0)
                        map_point[(roof_to_build[i][0],roof_to_build[i][1])]=roof_to_build[i]
                    
                    roof_P.append(roof_P[0])
                    roof=(create_my_roof(roof_P,roof_cells,rotation,radiant,roof_height))
                    result=STRUCT([result,T(3)(SIZE(3)(result)),roof])
                    return result
                else:
                    return result
                           
                return 0
            
            return add_roof
        
        return genrate_floors_stairs
    
    return generate_intern_structure


In [97]:
X=[0.3, 1.0, 0.2, 1.0, 0.2]
Y=[10, 5, 1, 5, 1, 5, 1, 5, 0.03125]
Z=[0.5]
mirror_or_not_mirror=[
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1]]
X2=[0.1, 0.5, 0.1, 0.5, 0.1, 0.5, 0.1]
Y2=[0.25, 1.25, 0.25, 1.25, 0.25, 1.25, 0.25, 1.25, 0.25]
Z2=[0.5]
mirror_or_not_mirror2=[
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1]]

doors_data=[X,Y,Z,mirror_or_not_mirror]
windows_data=[X2,Y2,Z,mirror_or_not_mirror2]
result=multisorey_house()(doors_data,windows_data,"lines_3b")(2)(1,[8,[[2,3,6,7],[1,8,7,2],[5,4,3,6]],[4,1,2,3,4,4,3,3],35])
VIEW(result)
result=multisorey_house()(doors_data,windows_data)(5)(1,[7,[[1,2,3,4],[4,5,6,7]],[3,3,4,1,2,3,3],35])
VIEW(result)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1ab7c1120> >

### Risultati 

#### esterni

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img1.png?raw=true)

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img2.png?raw=true)

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img5.png?raw=true)

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img6.png?raw=true)

#### interni

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img3.png?raw=true)

![casa](https://github.com/adixia/ggpl-LAB/blob/master/2017-1-13/pic/img4.png?raw=true)

